In [52]:
import ccxt
from datetime import datetime as dt,timedelta
import os
from pathlib import Path    
import pandas as pd
import numpy as np
import sys
import time
from tqdm import tqdm
import pandas_ta as ta
import vectorbt as vbt
import pytz

ex = ccxt.ftx({'enableRateLimit':True})

# Dowload index[0] to end_date
print('Investic Team Project',flush=True)

Investic Team Project


In [53]:
# get_data('LUNA-PERP',**kwarg)
def get_data(symbols:str  , **kwarg ):
    ''' Get Data By CCXT '''
    data= pd.DataFrame(ex.fetch_ohlcv(symbols,**kwarg ),columns=['date','open','high','low','close','volume'])
    data['date']=data['date'].apply(ex.iso8601)
    data['date']=pd.to_datetime(data['date'])
    data['date']=data['date'].dt.tz_localize(None)
    data.set_index('date',inplace=True)
    return data   
    
def download_tickers(tickers:list,**kwargs):
    ''' tickers = ticker on list '''
    all_data = dict()

    same_start = True
    for symbols in tqdm(tickers):
        print(symbols)
        df_ = get_data(symbols,**kwargs)

        if df_.empty:
            print(f'Cannot Get {symbols} Since {dt.fromtimestamp(int(since_ts/1000))} cant download')
            tf=  kwargs['timeframe'] 
            limit_ =  kwargs['limit'] 

            while df_.empty:
                #If cannot fetch since_ts Force to download last 1500 bars
                new_kwarg  = {'timeframe':tf,'limit':limit_}
                df_= get_data(symbols,**new_kwarg)
        all_data[symbols] =  df_
    print(f"[*] Download Complete  {tickers} \n")     

    return all_data

def download_same_start(all_data:dict,min_date:dict):
    ''' 
    Try to re-download with samestart date 
    all_data = OHLCV Data dict
    min_date = Min date on Data Dict'''
    # print(len(all_data))
    print(f"[i] Earliest Common Date {min_date['symbols']}: {min_date['min_date']}")
    same_start =True
    if same_start and len(all_data) > 1:
        tf = kwarg['timeframe']
        #Download data with same date
        result = {symbols:get_data(symbols,**{'since':int(min_date['min_date'].timestamp()*1000),'timeframe':tf}) for symbols,df in all_data.items()}

    else:
        result = all_data
    return result

In [54]:

def dtmask(df: pd.DataFrame, start: dt, end: dt):
    ''' SELECT DATA ON START END DATE RANGES'''
    return df.loc[(df.index >= start.strftime("%Y-%m-%d %H:%M:%S")) & (df.index <= end.strftime("%Y-%m-%d %H:%M:%S")), :].copy()

def check_duplicate(df):
    return df[df.index.duplicated()].copy()


def check_last_date(data:dict):
    ''' Check Last Index on DataFrame in Dictionary '''
    for sym ,df in new_ohlcv.items():
        print(sym, df.index[-1])



def get_min_date(data:dict):
    """Returns index of the earliest common index of all DataFrames in the dict"""
    min_date =  None
    symbol = None
    for symbols,df in data.items():
        if not df.empty:
            if min_date is None:
                symbol = symbols
                min_date = df.index[0]
            elif min_date < df.index[0]:
                symbol = symbols

                min_date = df.index[0]
                

    min_date = min_date.replace(tzinfo=None)
    return {'symbols':symbol,'min_date':min_date}    



In [55]:
def update_df(data:dict,end_date=dt,logs=True,**kwargs):
    ''' data : OHLCV dataframe on dict
        end_date = eg. datetime.now() datetime format '''
    new_data = {}

    for  symbol,df in data.items():
        if not df.empty:
            last_dt = df.index[-1].replace(tzinfo=None)
            last_ts = int(last_dt.timestamp()*1000)

            kwargs['since'] = last_ts
            new_df =get_data(symbol,**kwargs)
            update_df = pd.concat([df,new_df]).drop_duplicates()

            last_dt = update_df.index[-1].replace(tzinfo=None)
            last_ts = int(last_dt.timestamp()*1000)
            print(f'Original {symbol} DB Date {df.index[-1]}  : Update to {update_df.index[-1]}')
            print(f'Fetch Until {end_date}')
            while last_dt < end_date:
                kwargs['since'] = last_ts
                new_df =get_data(symbol,**kwargs)
                new_df_size_df  = new_df.shape[0] # Check size data 

                update_df = pd.concat([update_df,new_df]).drop_duplicates()
                last_dt = update_df.index[-1].replace(tzinfo=None)
                last_ts  =int(last_dt.timestamp()*1000)


                if (new_df_size_df <= 1 ) or ((new_df.empty)): # Check last data 
                    print(f'COMPLETE {symbol}')
                    print('='*100)
                    break    
                if logs:
                    print(f'Update  {new_df.index[0]}  :  {new_df.index[-1]} ') 
        update_df = update_df[~update_df.index.duplicated()]
        new_data[symbol] = update_df.sort_index()
        
    return new_data 

In [56]:
def fill_data(data:dict,end_date=dt,logs=True,**kwargs):
    new_data = {}
    for  symbol,df in data.items():
        if not df.empty: # Check DataFrame available
            first_dt = (df.index[0].replace(tzinfo=None)-timedelta(days=30))
            first_ts = int(first_dt.timestamp()*1000)

            kwargs['since'] = first_ts
            # print(kwargs)
            new_df =get_data(symbol,**kwargs)
            fill_data = pd.concat([df,new_df]).drop_duplicates().sort_index()

            first_dt = (fill_data.index[-1].replace(tzinfo=None)-timedelta(days=30))
            first_ts = int(first_dt.timestamp()*1000)
            print(f'FILL DATA {symbol} DB Date {df.index[0]}  : Update to {fill_data.index[0]}')
            print(end_date_fill,first_dt)
            while end_date_fill < first_dt:
                kwargs['since'] = first_ts
                new_df =get_data(symbol,**kwargs)
                new_df_size_df  = new_df.shape[0] # Check size data 
                fill_data = pd.concat([fill_data,new_df]).drop_duplicates().sort_index()
                first_dt = (fill_data.index[0].replace(tzinfo=None)-timedelta(days=30))
                first_ts  =int(first_dt.timestamp()*1000)

                if (new_df_size_df <= 1 ) or ((new_df.empty)):# or (new_df.index[0] ==first_dt): # Check last data 
                    print(f'COMPLETE {symbol}')
                    print('='*100)
                    break    
                if logs:
                    print(f'Update  {new_df.index[0]}  :  {fill_data.index[0]} , {new_df.shape[0]}') 
        fill_data = fill_data[~fill_data.index.duplicated()]
        new_data[symbol] = fill_data.sort_index()
    return new_data

In [47]:
df_test =pd.DataFrame()
df_test.shape[0]

0

In [48]:
# data = dowload_ticker(['ETH-PERP','LUNA/USD'],**{'since':since_ts,'timeframe':'1h'},same_start=True)

# Start with Selected Date

In [63]:
# First Install Data to dict
# tickers =['BTC/USD','ETH/USD','LUNA/USD','SPELL/USD']
tickers =['SPELL/USD','LUNA/USD']
tfx= '15m'
since_ =  dt(2020,6,1) #dt.now() -timedelta(days=180)
since_ts = int(since_.timestamp()*1000)
kwarg = {'since':since_ts,'timeframe':tfx,'limit':500}

ohlcv = download_tickers(tickers,**kwarg)
# ohlcv['LUNA/USD']


  0%|                                                                                                                     | 0/2 [00:00<?, ?it/s]

SPELL/USD
Cannot Get SPELL/USD Since 2020-06-01 00:00:00 cant download


 50%|██████████████████████████████████████████████████████▌                                                      | 1/2 [00:00<00:00,  2.62it/s]

LUNA/USD
Cannot Get LUNA/USD Since 2020-06-01 00:00:00 cant download


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.74it/s]

[*] Download Complete  ['SPELL/USD', 'LUNA/USD'] 



In [64]:
ohlcv['LUNA/USD']

,open,high,low,close,volume
date,,,,,
2022-03-28 07:45:00,95.49,95.71,95.43,95.58,375990.8002
2022-03-28 08:00:00,95.58,95.73,95.10,95.10,153124.6543
2022-03-28 08:15:00,95.10,95.44,94.62,95.38,118958.1919
2022-03-28 08:30:00,95.38,95.46,94.78,94.87,396878.3284
2022-03-28 08:45:00,94.87,95.13,94.84,95.01,59344.6264
...,...,...,...,...,...
2022-04-02 11:30:00,111.77,112.20,111.66,112.20,134211.1071
2022-04-02 11:45:00,112.20,112.23,111.28,111.52,242603.0763
2022-04-02 12:00:00,111.52,111.99,110.72,111.39,347402.1868


# เติม Data ย้อนหลังถึง XXX (datetime)

In [65]:
end_date_fill = dt(2020,1,1) # End Fill datetime
kwarg = {'timeframe':tfx,'limit':500}

ohlcv = fill_data(ohlcv,end_date_fill,**kwarg)
check_duplicate(ohlcv['LUNA/USD'])

FILL DATASPELL/USD DB Date 2022-03-28 07:45:00  : Update to 2022-02-26 07:45:00
2020-01-01 00:00:00 2022-03-03 12:30:00
Update  2022-03-03 12:30:00  :  2022-02-26 07:45:00 , 500
Update  2022-01-27 07:45:00  :  2022-01-27 07:45:00 , 500
Update  2021-12-28 07:45:00  :  2021-12-28 07:45:00 , 500
Update  2021-11-28 07:45:00  :  2021-11-28 07:45:00 , 500
Update  2021-10-29 07:45:00  :  2021-10-29 07:45:00 , 500
COMPLETE SPELL/USD
FILL DATALUNA/USD DB Date 2022-03-28 07:45:00  : Update to 2022-03-01 14:00:00
2020-01-01 00:00:00 2022-03-03 12:30:00
Update  2022-03-03 12:30:00  :  2022-03-01 14:00:00 , 500
COMPLETE LUNA/USD


,open,high,low,close,volume
date,,,,,


In [61]:
ohlcv['BTC/USD']

,open,high,low,close,volume
date,,,,,
2020-01-02 17:00:00,7021.5,7021.5,6903.5,6944.0,243276.14780
2020-01-02 17:15:00,6944.0,6952.0,6903.5,6919.0,87080.39155
2020-01-02 17:30:00,6919.0,6954.0,6919.0,6947.5,7416.24265
2020-01-02 17:45:00,6946.0,6948.5,6933.5,6941.5,1565.53475
2020-01-02 18:00:00,6941.5,6941.5,6924.0,6936.5,8408.79335
...,...,...,...,...,...
2020-06-05 20:45:00,9687.0,9713.0,9686.5,9713.0,4027.60180
2020-06-05 21:00:00,9713.0,9718.0,9688.5,9691.0,23652.36295
2020-06-05 21:15:00,9691.0,9700.5,9683.5,9699.0,4861.12995


# Update Data Until xxx (Datetime)

In [66]:
# update until nows
end_date = dt.now()
kwarg = {'timeframe':tfx,'limit':500}

ohlcv =update_df(ohlcv,end_date=end_date,**kwarg) # not same start
check_duplicate(ohlcv['SPELL/USD'])

Original SPELL/USD DB Date 2022-04-02 12:30:00  : Update to 2022-04-02 12:30:00
Fetch Until 2022-04-02 19:39:49.091913
COMPLETE SPELL/USD
Original LUNA/USD DB Date 2022-04-02 12:30:00  : Update to 2022-04-02 12:30:00
Fetch Until 2022-04-02 19:39:49.091913
COMPLETE LUNA/USD


,open,high,low,close,volume
date,,,,,


In [67]:
# END  , TEST Merge all data to One DataFrame

df_all = pd.DataFrame()
for symbol ,df in ohlcv.items():
    df = ohlcv[symbol]
    df['symbol'] = symbol
    df_all =df_all.append(df)

In [68]:
df_all.sort_index()

,open,high,low,close,volume,symbol
date,,,,,,
2021-10-29 07:45:00,0.025132,0.025132,0.024733,0.024745,67805.13075,SPELL/USD
2021-10-29 08:00:00,0.024745,0.025000,0.024740,0.024823,24215.04100,SPELL/USD
2021-10-29 08:15:00,0.024823,0.024932,0.024700,0.024775,21874.66550,SPELL/USD
2021-10-29 08:30:00,0.024775,0.024905,0.024480,0.024575,23338.01150,SPELL/USD
2021-10-29 08:45:00,0.024575,0.025365,0.024243,0.024872,265711.03000,SPELL/USD
...,...,...,...,...,...,...
2022-04-02 12:00:00,111.520000,111.990000,110.720000,111.390000,347402.18680,LUNA/USD
2022-04-02 12:15:00,111.390000,111.950000,111.250000,111.660000,252114.83000,LUNA/USD
2022-04-02 12:15:00,0.004610,0.004645,0.004610,0.004642,26487.57500,SPELL/USD


In [69]:
# Check Data
all_data =  []
for sym , df in ohlcv.items():
    cum_ret = (df['close'].pct_change()+1).cumprod()-1
    final_ret  =  cum_ret[-1]
    data = {'symbol':sym,'start_price':df['close'].iloc[0],'start_index':df.index[0],\
            'end_index':df.index[-1],'end_price':df['close'].iloc[-1],'ret':final_ret,'bars':   df.shape[0]}
    all_data.append(data)
    
check_df = pd.DataFrame(all_data)
check_df   


,symbol,start_price,start_index,end_index,end_price,ret,bars
0,SPELL/USD,0.024745,2021-10-29 07:45:00,2022-04-02 12:30:00,0.004727,-0.808951,3499
1,LUNA/USD,91.770000,2022-03-01 14:00:00,2022-04-02 12:30:00,112.290000,0.223602,1186


# Same Start Date Download

In [70]:
# First Install Data to dict
# tickers =['BTC/USD','ETH/USD','LUNA/USD','SPELL/USD']
tickers =['BTC/USD','ETH/USD','SPELL/USD','LUNA/USD']
tfx= '15m'
since_ =  dt(2020,6,1) #dt.now() -timedelta(days=180)
since_ts = int(since_.timestamp()*1000)
kwarg = {'since':since_ts,'timeframe':tfx,'limit':500}

new_ohlcv = download_tickers(tickers,**kwarg)
# ohlcv['LUNA/USD']
# check_duplicate(ohlcv['LUNA/USD'])

  0%|                                                                                                                     | 0/4 [00:00<?, ?it/s]

BTC/USD


 25%|███████████████████████████▎                                                                                 | 1/4 [00:00<00:00,  3.25it/s]

ETH/USD


 50%|██████████████████████████████████████████████████████▌                                                      | 2/4 [00:00<00:00,  3.24it/s]

SPELL/USD
Cannot Get SPELL/USD Since 2020-06-01 00:00:00 cant download


 75%|█████████████████████████████████████████████████████████████████████████████████▊                           | 3/4 [00:01<00:00,  2.95it/s]

LUNA/USD
Cannot Get LUNA/USD Since 2020-06-01 00:00:00 cant download


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.93it/s]

[*] Download Complete  ['BTC/USD', 'ETH/USD', 'SPELL/USD', 'LUNA/USD'] 



# Show Min Date on OHLCV Dictionary and restart download with min_date

In [71]:
min_date = get_min_date(ohlcv)
print('Download ALL SYMBOLS @' ,min_date['min_date'])
new_ohlcv =download_same_start(new_ohlcv,min_date)
check_last_date(new_ohlcv)

Download ALL SYMBOLS @ 2022-03-01 14:00:00
[i] Earliest Common Date LUNA/USD: 2022-03-01 14:00:00
BTC/USD 2022-03-17 05:00:00
ETH/USD 2022-03-17 05:00:00
SPELL/USD 2022-03-17 05:00:00
LUNA/USD 2022-03-17 05:00:00


In [75]:
# new_ohlcv['SPELL/USD']

,open,high,low,close,volume
date,,,,,
2022-03-01 14:00:00,0.004743,0.004755,0.004723,0.004740,23901.31675
2022-03-01 14:15:00,0.004740,0.005000,0.004723,0.004830,370338.35600
2022-03-01 14:30:00,0.004830,0.004882,0.004700,0.004783,105072.45275
2022-03-01 14:45:00,0.004783,0.004788,0.004640,0.004675,115351.55750
2022-03-01 15:00:00,0.004675,0.004707,0.004618,0.004630,46661.03700
...,...,...,...,...,...
2022-03-17 04:00:00,0.003478,0.003525,0.003465,0.003525,10149.90250
2022-03-17 04:15:00,0.003525,0.003533,0.003510,0.003530,3049.42100
2022-03-17 04:30:00,0.003530,0.003530,0.003515,0.003515,6601.87650


In [76]:
# update untill now
end_date = dt.now()
kwarg = {'timeframe':'1h','limit':500}
new_ohlcv=  update_df(new_ohlcv,end_date=end_date,**kwarg) # Same start


Original BTC/USD DB Date 2022-03-17 05:00:00  : Update to 2022-04-02 12:00:00
Fetch Until 2022-04-02 19:41:21.084480
COMPLETE BTC/USD
Original ETH/USD DB Date 2022-03-17 05:00:00  : Update to 2022-04-02 12:00:00
Fetch Until 2022-04-02 19:41:21.084480
COMPLETE ETH/USD
Original SPELL/USD DB Date 2022-03-17 05:00:00  : Update to 2022-04-02 12:00:00
Fetch Until 2022-04-02 19:41:21.084480
COMPLETE SPELL/USD
Original LUNA/USD DB Date 2022-03-17 05:00:00  : Update to 2022-04-02 12:00:00
Fetch Until 2022-04-02 19:41:21.084480
COMPLETE LUNA/USD


In [77]:
end_date_fill = dt(2020,1,1)
kwarg = {'timeframe':tfx,'limit':500}

new_ohlcv = fill_data(new_ohlcv,end_date_fill,**kwarg)
check_duplicate(new_ohlcv['SPELL/USD'])

FILL DATABTC/USD DB Date 2022-03-01 14:00:00  : Update to 2022-01-30 14:00:00
2020-01-01 00:00:00 2022-03-03 12:00:00
Update  2022-03-03 12:00:00  :  2022-01-30 14:00:00 , 500
Update  2021-12-31 14:00:00  :  2021-12-31 14:00:00 , 500
Update  2021-12-01 14:00:00  :  2021-12-01 14:00:00 , 500
Update  2021-11-01 14:00:00  :  2021-11-01 14:00:00 , 500
Update  2021-10-02 14:00:00  :  2021-10-02 14:00:00 , 500
Update  2021-09-02 14:00:00  :  2021-09-02 14:00:00 , 500
Update  2021-08-03 14:00:00  :  2021-08-03 14:00:00 , 500
Update  2021-07-04 14:00:00  :  2021-07-04 14:00:00 , 500
Update  2021-06-04 14:00:00  :  2021-06-04 14:00:00 , 500
Update  2021-05-05 14:00:00  :  2021-05-05 14:00:00 , 500
Update  2021-04-05 14:00:00  :  2021-04-05 14:00:00 , 500
Update  2021-03-06 14:00:00  :  2021-03-06 14:00:00 , 500
Update  2021-02-04 14:00:00  :  2021-02-04 14:00:00 , 500
Update  2021-01-05 14:00:00  :  2021-01-05 14:00:00 , 500
Update  2020-12-06 14:00:00  :  2020-12-06 14:00:00 , 500
Update  2020

,open,high,low,close,volume
date,,,,,


In [79]:
# new_ohlcv

# DB ZONE
## DataFrame save to DB (postgres,sqlite,mongodb)

In [80]:
import re
def split_sym_name(symbols:str):
    ''' Replace _ to / and - on symbol_name   '''
    if len(symbols.split('/')) >=2 :
        symbol = symbols.split('/')
        
    elif len(symbols.split('-')) >=2:
        symbol = symbols.split('-')
        
    else:
        symbol = symbols

    return symbol


def _edit_symbol(list_str:list,sym_sign ='/'):
    # Remove sign before save db
    " APPEND SYMBOL SIGN TO STR ON LIST"
    
    symbols = []
    for s in list_str:
        x =re.search("USD$", s)
        print(x)
        if x:
            pos =x.regs[0][0]
            symbol = s[0:pos] +sym_sign + s[pos:]
            symbols.append(symbol)
            
    return symbols



# Test
symbols_list_test = ['ETHUSD','BTCUSD','AXSUSD','BTC-PERP']
_edit_symbol(list_str =symbols_list_test)

<re.Match object; span=(3, 6), match='USD'>
<re.Match object; span=(3, 6), match='USD'>
<re.Match object; span=(3, 6), match='USD'>
None


['ETH/USD', 'BTC/USD', 'AXS/USD']